In [ ]:
%pip install --upgrade tiktoken
%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [ ]:
import tiktoken
import openai
import pandas as pd

model_encoding_dict={
    'cl100k_base':	['GPT-3.5-Turbo-4k', 'GPT-3.5-Turbo-16k', 'GPT-3.5-Turbo-1106', 'GPT-4-Turbo', 'GPT-4-Turbo-Vision', 'GPT-4-8k', 'GPT-4-32k', 'text-embedding-ada-002'],
    'p50k_base'	:['text-davinci-002', 'text-davinci-003'],
}

azure_costing= {
    'Models': ['GPT-3.5-Turbo-4k', 'GPT-3.5-Turbo-16k', 'GPT-3.5-Turbo-1106', 'GPT-4-Turbo', 'GPT-4-Turbo-Vision', 'GPT-4-8k', 'GPT-4-32k', 'Babbage-002', 'Davinci-002'],
    'Prompt': [0.0015, 0.003, 0, 0.01, 0.01, 0.03, 0.06, 0.0004, 0.002],
    'Completion': [0.002, 0.004, 0, 0.03, 0.03, 0.06, 0.12, 0, 0],
    'Usage': [0, 0, 0, 0, 0, 0, 0, 0.0004, 0.002]
}

azure_costing_df = pd.DataFrame(azure_costing)

def tiktoken_setup():
  encoding = tiktoken.get_encoding("cl100k_base")
  encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def num_tokens_from_string(string: str, encoding_name: str) -> int:
  """Returns the number of tokens in a text string."""
  encoding = tiktoken.get_encoding(encoding_name)
  num_tokens = len(encoding.encode(string))
  return num_tokens

def map_model_encoding(model: str):
  for key in model_encoding_dict:
    if model in model_encoding_dict[key]:
      encoding=key
      print('Model found - using encoding model ', key)
  return encoding

def costing(model, input_tokens, output_tokens):
  result = azure_costing_df[azure_costing_df['Models'] == model]
  print(result)
  if model=='Babbage-002' or model=='Davinci-002':
    cost=result['Usage']*((input_tokens+output_tokens)/1000)
  else:
    cost=(result['Prompt'].values[0]*(input_tokens/1000))+(result['Completion'].values[0]*(output_tokens/1000))
    print('prompt: ', result['Prompt'].values[0], 'tokens: ', input_tokens, 'completion: ', result['Completion'], 'output: ', output_tokens)

  print(cost)



def main():
  tiktoken_setup()
  # model=input("What is the model you are using: ")
  model='GPT-4-8k'
  encoding=map_model_encoding(model)
  print(encoding)
  # sample_input=input("Please enter the last input used in your GPT model: ")
  # sample_output=input("Please enter the output generated for the entered input: ")
  sample_input="""
  turn the following data into a pandas df


Language models
Models 	Context 	Prompt (Per 1,000 tokens) 	Completion (Per 1,000 tokens)
GPT-3.5-Turbo 	4K 	$0.0015 	$0.002
GPT-3.5-Turbo 	16K 	$0.003 	$0.004
GPT-3.5-Turbo-1106 	16K 	N/A 	N/A
GPT-4-Turbo 	128K 	$0.01 	$0.03
GPT-4-Turbo-Vision 	128K 	$0.01 	$0.03
GPT-4 	8K 	$0.03 	$0.06
GPT-4 	32K 	$0.06 	$0.12
Base models
Models 	Usage per 1,000 tokens
Babbage-002 	$0.0004
Davinci-002 	$0.002
  """
  sample_output="""
  You can create a pandas DataFrame with the provided data using the following code:

```python
import pandas as pd

data = {
    'Models': ['GPT-3.5-Turbo', 'GPT-3.5-Turbo', 'GPT-3.5-Turbo-1106', 'GPT-4-Turbo', 'GPT-4-Turbo-Vision', 'GPT-4', 'GPT-4', 'Babbage-002', 'Davinci-002'],
    'Context': [4_000, 16_000, 16_000, 128_000, 128_000, 8_000, 32_000, None, None],
    'Prompt (Per 1,000 tokens)': [0.0015, 0.003, None, 0.01, 0.01, 0.03, 0.06, 0.0004, 0.002],
    'Completion (Per 1,000 tokens)': [0.002, 0.004, None, 0.03, 0.03, 0.06, 0.12, None, None],
    'Usage per 1,000 tokens': [None, None, None, None, None, None, None, 0.0004, 0.002]
}

df = pd.DataFrame(data)
print(df)
```

This will create a DataFrame with the provided data, and you can use it for further analysis or manipulation in your Python environment.
  """
  input_tokens=num_tokens_from_string(sample_input, encoding)
  output_tokens=num_tokens_from_string(sample_output, encoding)
  costing(model, input_tokens, output_tokens)



In [ ]:
main()

Model found - using encoding model  cl100k_base
cl100k_base
     Models  Prompt  Completion  Usage
5  GPT-4-8k    0.03        0.06    0.0
prompt:  0.03 tokens:  246 completion:  5    0.06
Name: Completion, dtype: float64 output:  332
0.027299999999999998


In [ ]:
num_tokens_from_string("tiktoken is great!", "cl100k_base")

6